### 1. Chicago Socioeconomic Indicators
This <a ref="https://data.cityofchicago.org/Health-Human-Services/Census-Data-Selected-socioeconomic-indicators-in-C/kn9c-c2s2" >dataset</a> contains a selection of six socioeconomic indicators of public health significance and a “hardship index,” by Chicago community area, for the years 2008 – 2012.

### 2. Chicago Public Schools
This <a ref="https://data.cityofchicago.org/Education/Chicago-Public-Schools-Progress-Report-Cards-2011-/9xs2-f89t">dataset</a> shows all school level performance data used to create CPS School Report Cards for the 2011-2012 school year.

### 3. Chicago Crime Data
This <a ref="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2">dataset</a> reflects reported incidents of crime (with the exception of murders where data exists for each victim) that occurred in the City of Chicago from 2001 to present, minus the most recent seven days.

Chicago Socioeconomic Indicators: https://ibm.box.com/shared/static/05c3415cbfbtfnr2fx4atenb2sd361ze.csv

Chicago Public Schools: https://ibm.box.com/shared/static/f9gjvj1gjmxxzycdhplzt01qtz0s7ew7.csv

Chicago Crime Data: https://ibm.box.com/shared/static/svflyugsr9zbqy5bmowgswqemfpm1x7f.csv

In [1]:
import ibm_db
import pandas
import ibm_db_dbi

In [17]:
#Replace the placeholder values with your actual Db2 hostname, username, and password:
dsn_hostname = "dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net" # e.g.: "dashdb-txn-sbox-yp-dal09-04.services.dal.bluemix.net"
dsn_uid = "xqn45df297"        # e.g. "abc12345"
dsn_pwd = "461654KJKJjkk@153"      # e.g. "7dBZ3wWt9XN6$o0J"

dsn_driver = "{IBM DB2 ODBC DRIVER}"
dsn_database = "BLUDB"            # e.g. "BLUDB"
dsn_port = "50000"                # e.g. "50000" 
dsn_protocol = "TCPIP"            # i.e. "TCPIP"

In [3]:
#DO NOT MODIFY THIS CELL. Just RUN it with Shift + Enter
#Create the dsn connection string
dsn = (
    "DRIVER={0};"
    "DATABASE={1};"
    "HOSTNAME={2};"
    "PORT={3};"
    "PROTOCOL={4};"
    "UID={5};"
    "PWD={6};").format(dsn_driver, dsn_database, dsn_hostname, dsn_port, dsn_protocol, dsn_uid, dsn_pwd)

#print the connection string to check correct values are specified
print(dsn)

DRIVER={IBM DB2 ODBC DRIVER};DATABASE=BLUDB;HOSTNAME=dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net;PORT=50000;PROTOCOL=TCPIP;UID=xqn45297;PWD=kakadiyaKamal@153;


In [4]:
try:
    conn = ibm_db.connect(dsn, "", "")
    print ("Connected to database: ", dsn_database, "as user: ", dsn_uid, "on host: ", dsn_hostname)

except:
    print ("Unable to connect: ", ibm_db.conn_errormsg() )

('Connected to database: ', 'BLUDB', 'as user: ', 'xqn45297', 'on host: ', 'dashdb-txn-sbox-yp-lon02-02.services.eu-gb.bluemix.net')


In [5]:
#Retrieve Metadata for the Database Server
server = ibm_db.server_info(conn)

print ("DBMS_NAME: ", server.DBMS_NAME)
print ("DBMS_VER:  ", server.DBMS_VER)
print ("DB_NAME:   ", server.DB_NAME)

('DBMS_NAME: ', 'DB2/LINUXX8664')
('DBMS_VER:  ', '11.01.0404')
('DB_NAME:   ', 'BLUDB')


## 1. Find the total number of crimes recorded in the Crime table

In [6]:
query='select count(*) from CHICAGO_CRIME_DATA';
pconn = ibm_db_dbi.Connection(conn)
result = pandas.read_sql(query, pconn)
print(result)

       1
0  533.0


## 2. Retrieve first 10 rows from the CRIME table

In [7]:
query='select * from CHICAGO_CRIME_DATA fetch first 10 rows only';
pconn = ibm_db_dbi.Connection(conn)
result = pandas.read_sql(query, pconn) print(result)

        ID CASE_NUMBER                    DATE                   BLOCK  IUCR  \
0    21149    HW519443  11/03/2013 07:27:00 PM     044XX S RICHMOND ST   110   
1    23469    JA359626  07/23/2017 09:25:00 AM         015XX E 82ND ST   110   
2  1326195     G021609  01/11/2001 02:30:41 AM     087XX S ESCANABA AV  9901   
3  1340847     G040244  01/19/2001 06:39:03 PM        063XX N NAGLE AV   820   
4  1353618     G056330  01/27/2001 04:20:00 PM       078XX S SAWYER AV   460   
5  1363954     G070193  02/03/2001 03:00:00 AM   004XX W WRIGHTWOOD AV   460   
6  1367327     G057394  01/28/2001 07:10:00 AM       046XX S CICERO AV  1513   
7  1414626     G134016  03/01/2001 11:00:00 PM   055XX S NOTTINGHAM AV  1310   
8  1419496     G140454  03/11/2001 04:44:05 PM  077XX S SOUTH SHORE DR   460   
9  1427912     G122095  03/02/2001 04:20:00 PM      039XX N ASHLAND AV  1505   

      PRIMARY_TYPE           DESCRIPTION     LOCATION_DESCRIPTION ARREST  \
0         HOMICIDE   FIRST DEGREE MURDER   

## 3. How many crimes involve an arrest

In [8]:
query='select count(1) from CHICAGO_CRIME_DATA where arrest = "TRUE"';
pconn = ibm_db_dbi.Connection(conn)
result = pandas.read_sql(query, pconn)
print(result)

       1
0  163.0


## 4. Which unique types of crimes have been recorded at a GAS STATION?

In [9]:
query="select distinct PRIMARY_TYPE from CHICAGO_CRIME_DATA where LOCATION_DESCRIPTION = 'GAS STATION'";
pconn = ibm_db_dbi.Connection(conn)
result = pandas.read_sql(query, pconn)
print(result)

      PRIMARY_TYPE
0  CRIMINAL TRESPA
1        NARCOTICS
2          ROBBERY
3            THEFT


## 5. In the CENUS_DATA table list all community areas whose names start with the letter ‘B’.

In [10]:
query="select COMMUNITY_AREA_NAME from CENSUS_DATA where COMMUNITY_AREA_NAME like 'B%'";
pconn = ibm_db_dbi.Connection(conn)
result = pandas.read_sql(query, pconn)
print(result)

  COMMUNITY_AREA_NAME
0      Belmont Cragin
1            Burnside
2       Brighton Park
3          Bridgeport
4             Beverly


## 6. Which schools in Community Areas 10 to 15 are healthy school certified?

In [11]:
query="SELECT NAME_OF_SCHOOL FROM CHICAGO_PUBLIC_SCHOOLS WHERE HEALTHY_SCHOOL_CERTIFIED = 'Yes' \
AND  COMMUNITY_AREA_NUMBER BETWEEN 10 and 15";
pconn = ibm_db_dbi.Connection(conn)
result = pandas.read_sql(query, pconn)
print(result)

                    NAME_OF_SCHOOL
0  Rufus M Hitch Elementary School


## 7. What is the average school Safety Score?

In [12]:
query="select avg(SAFETY_SCORE) as avg_school_safety from CHICAGO_PUBLIC_SCHOOLS";
pconn = ibm_db_dbi.Connection(conn)
result = pandas.read_sql(query, pconn)
print(result)

   AVG_SCHOOL_SAFETY
0          49.504873


## 8. List the top 5 Community Areas by average College Enrollments [number of students]

In [13]:
query="select COMMUNITY_AREA_NAME , avg(COLLEGE_ENROLLMENT) as AVG_ENROLLMENT from CHICAGO_PUBLIC_SCHOOLS group by \
COMMUNITY_AREA_NAME order by AVG_ENROLLMENT desc limit 5";
pconn = ibm_db_dbi.Connection(conn)
result = pandas.read_sql(query, pconn)
print(result)

  COMMUNITY_AREA_NAME  AVG_ENROLLMENT
0            NDA+T388     1053.000000
1             Average      696.900000
2                Weak      667.482758
3                 NDA      603.817164
4              Strong      567.593750


## 9. Use a sub-query to determine which Community Area has the least value for Safety Score?

In [14]:
query="select COMMUNITY_AREA_NAME from CHICAGO_PUBLIC_SCHOOLS where SAFETY_SCORE = \
(select min(SAFETY_SCORE) from CHICAGO_PUBLIC_SCHOOLS)";
pconn = ibm_db_dbi.Connection(conn)
result = pandas.read_sql(query, pconn)
print(result)

  COMMUNITY_AREA_NAME
0                 NDA


## 10. Find the Per Capita Income of the Community Area which has a school Safety Score of 1.

In [15]:
query="select A.PER_CAPITA_INCOME from CENSUS_DATA A, CHICAGO_PUBLIC_SCHOOLS B where A.COMMUNITY_AREA_NUMBER \
= B.COMMUNITY_AREA_NUMBER and B.SAFETY_SCORE = 1";
pconn = ibm_db_dbi.Connection(conn)
result = pandas.read_sql(query, pconn)
print(result)

   PER_CAPITA_INCOME
0              13785


In [16]:
ibm_db.close(conn)

True